# 0. Install and import required libraries

In [501]:
!pip install selenium
!pip install html5lib

In [502]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import io

# 1. Web scraping and consolidation functions

**Scraping Function**

In [1020]:
def scrape_fbref_stats(url, table_index=2):
    """
    Scraps fbref urls defined as function input and brings table number 2 (expected 'Player Stats') as output

    Parameters:
    - url: link of the url to scrap
    - table_index: defined as a fixed number '2' as all pages seem to obbey to similar logic 
    
    Returns:
    - Dataframe with scraped table
    """
    
    # Set up headless browser
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    # Load the page
    driver.get(url)
    time.sleep(5)  # Wait for JavaScript to render

    # Get HTML content after rendering
    html = driver.page_source
    driver.quit()

    # Parse with pandas
    tables = pd.read_html(io.StringIO(html))
    df = tables[table_index]

    # Flatten multi-index headers properly
    df.columns = [
        f'{col[0]}_{col[1]}' if isinstance(col, tuple) and col[0] != col[1] and 'level' not in col[0]
        else col[1] if isinstance(col, tuple)
        else col
        for col in df.columns
]

    # Reset index
    df = df.reset_index(drop=True)

    # No truncation: keep full names
    df.columns = [str(col).strip() for col in df.columns]

    # Fill missing values and discart invalid rows
    df = df.fillna(0)
    df = df[df['Rk'] != 'Rk']

    # Simplify Nation field if it exists
    if 'Nation' in df.columns:
        df['Nation'] = df['Nation'].astype(str).str.split(' ').str.get(1)

    return df

**Player Aggregation Function**

In [1022]:
def join_player_tables(tables, league_name = None):
    """
    Joins a list of DataFrames on 'Rk' and 'Player' columns using inner join.

    Parameters:
    - tables: list of pandas DataFrames to be joined
    - league_name: optional string, added as a 'League' column

    Returns:
    - A single DataFrame resulting from joining all tables
    """
    if not tables:
        return pd.DataFrame()  # Return empty if list is empty

    # Start with the first table
    merged_df = tables[0]

    for i, df in enumerate(tables[1:], start=2):
        merged_df = pd.merge(merged_df, df, on=['Rk', 'Player'], how='inner', suffixes=('', f'_{i}'))

    # Add and move 'League' column to the front
    if league_name is not None:
        merged_df['League'] = league_name
        cols = ['League'] + [col for col in merged_df.columns if col != 'League']
        merged_df = merged_df[cols]
    
    return merged_df

# 2. National Leagues Web Scraping

## Austria

### AUS 1 - Standard Stats, Shooting and Miscellaneous

In [1026]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/56/stats/Austrian-Bundesliga-Stats"
aus1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/56/shooting/Austrian-Bundesliga-Stats"
aus1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/56/misc/Austrian-Bundesliga-Stats"
aus1_misc = scrape_fbref_stats(url3)

In [1027]:
# Aggregate data
aus1_agg = join_player_tables([aus1_stand, aus1_shooting, aus1_misc], 'aus1')

# Check number of columns and rows
print(aus1_stand.shape, aus1_shooting.shape, aus1_misc.shape)
aus1_agg.shape

(358, 25) (358, 20) (358, 21)


(358, 63)

## Belgium

### BEL 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1030]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/37/stats/Belgian-Pro-League-Stats"
bel1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/37/shooting/Belgian-Pro-League-Stats"
bel1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/37/misc/Belgian-Pro-League-Stats"
bel1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/37/passing/Belgian-Pro-League-Stats"
bel1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/37/defense/Belgian-Pro-League-Stats"
bel1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/37/possession/Belgian-Pro-League-Stats"
bel1_pos= scrape_fbref_stats(url6)

In [1031]:
# Aggregate data
bel1_agg = join_player_tables([bel1_stand, bel1_shooting, bel1_misc, bel1_pass, bel1_def, bel1_pos], 'bel1')

# Check number of columns and rows
print(bel1_stand.shape, bel1_shooting.shape, bel1_misc.shape, bel1_pass.shape, bel1_def.shape, bel1_pos.shape)
bel1_agg.shape

(486, 37) (486, 26) (486, 25) (486, 32) (486, 25) (486, 31)


(486, 167)

### BEL 2 - Standard Stats, Shooting and Miscellaneous

In [1033]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/69/stats/Challenger-Pro-League-Stats"
bel2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/69/shooting/Challenger-Pro-League-Stats"
bel2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/69/misc/Challenger-Pro-League-Stats"
bel2_misc = scrape_fbref_stats(url3)

In [1034]:
# Aggregate data
bel2_agg = join_player_tables([bel2_stand, bel2_shooting, bel2_misc], 'bel2')

# Check number of columns and rows
print(bel2_stand.shape, bel2_shooting.shape, bel2_misc.shape)
bel2_agg.shape

(459, 25) (459, 20) (459, 21)


(459, 63)

## Bulgaria

### BUL 1 - Standard Stats and Miscellaneous

In [1260]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/67/2024-2025/stats/2024-2025-Bulgarian-First-League-Stats"
bul1_stand = scrape_fbref_stats(url1)
# SHOOTING - NOT AVAILABLE
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/67/2024-2025/misc/2024-2025-Bulgarian-First-League-Stats"
bul1_misc = scrape_fbref_stats(url3)

In [1261]:
# Aggregate data
bul1_agg = join_player_tables([bul1_stand, bul1_misc], 'bul1')

# Check number of columns and rows
print(bul1_stand.shape, bul1_misc.shape)
bul1_agg.shape

(556, 25) (556, 21)


(556, 45)

## Croatia

### CRO 1 - Standard Stats, Shooting and Miscellaneous

In [1037]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/63/stats/Hrvatska-NL-Stats"
cro1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/63/shooting/Hrvatska-NL-Stats"
cro1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/63/misc/Hrvatska-NL-Stats"
cro1_misc = scrape_fbref_stats(url3)

In [1038]:
# Aggregate data
cro1_agg = join_player_tables([cro1_stand, cro1_shooting, cro1_misc], 'cro1')

# Check number of columns and rows
print(cro1_stand.shape, cro1_shooting.shape, cro1_misc.shape)
cro1_agg.shape

(346, 25) (346, 20) (346, 21)


(346, 63)

## Czech Republic

### CZE 1 - Standard Stats, Shooting and Miscellaneous

In [1041]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/66/2024-2025/stats/2024-2025-Czech-First-League-Stats"
cze1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/66/2024-2025/shooting/2024-2025-Czech-First-League-Stats"
cze1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/66/2024-2025/misc/2024-2025-Czech-First-League-Stats"
cze1_misc = scrape_fbref_stats(url3)

In [1042]:
# Aggregate data
cze1_agg = join_player_tables([cze1_stand, cze1_shooting, cze1_misc], 'cze1')

# Check number of columns and rows
print(cze1_stand.shape, cze1_shooting.shape, cze1_misc.shape)
cze1_agg.shape

(545, 25) (545, 20) (545, 21)


(545, 63)

## Denmark

### DEN 1 - Standard Stats, Shooting and Miscellaneous

In [1045]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/50/2024-2025/stats/2024-2025-Danish-Superliga-Stats"
den1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/50/2024-2025/shooting/2024-2025-Danish-Superliga-Stats"
den1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/50/2024-2025/misc/2024-2025-Danish-Superliga-Stats"
den1_misc = scrape_fbref_stats(url3)

In [1046]:
# Aggregate data
den1_agg = join_player_tables([den1_stand, den1_shooting, den1_misc], 'den1')

# Check number of columns and rows
print(den1_stand.shape, den1_shooting.shape, den1_misc.shape)
den1_agg.shape

(370, 25) (370, 20) (370, 21)


(370, 63)

## England

### ENG 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1049]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/9/stats/Premier-League-Stats"
eng1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/9/shooting/Premier-League-Stats"
eng1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/9/misc/Premier-League-Stats"
eng1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/9/passing/Premier-League-Stats"
eng1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/9/defense/Premier-League-Stats"
eng1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/9/possession/Premier-League-Stats"
eng1_pos= scrape_fbref_stats(url6)

In [1050]:
# Aggregate data
eng1_agg = join_player_tables([eng1_stand, eng1_shooting, eng1_misc, eng1_pass, eng1_def, eng1_pos], 'eng1')

# Check number of columns and rows
print(eng1_stand.shape, eng1_shooting.shape, eng1_misc.shape, eng1_pass.shape, eng1_def.shape, eng1_pos.shape)
eng1_agg.shape

(574, 37) (574, 26) (574, 25) (574, 32) (574, 25) (574, 31)


(574, 167)

### ENG 2 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1052]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/10/stats/Championship-Stats"
eng2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/10/shooting/Championship-Stats"
eng2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/10/misc/Championship-Stats"
eng2_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/10/passing/Championship-Stats"
eng2_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/10/defense/Championship-Stats"
eng2_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/10/possession/Championship-Stats"
eng2_pos= scrape_fbref_stats(url6)

In [1053]:
# Aggregate data
eng2_agg = join_player_tables([eng2_stand, eng2_shooting, eng2_misc, eng2_pass, eng2_def, eng2_pos], 'eng2')

# Check number of columns and rows
print(eng2_stand.shape, eng2_shooting.shape, eng2_misc.shape, eng2_pass.shape, eng2_def.shape, eng2_pos.shape)
eng2_agg.shape

(765, 37) (765, 26) (765, 25) (765, 32) (765, 25) (765, 31)


(765, 167)

### ENG 3 - Standard Stats, Shooting and Miscellaneous

In [1055]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/15/stats/League-One-Stats"
eng3_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/15/shooting/League-One-Stats"
eng3_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/15/misc/League-One-Stats"
eng3_misc = scrape_fbref_stats(url3)

In [1056]:
# Aggregate data
eng3_agg = join_player_tables([eng3_stand, eng3_shooting, eng3_misc], 'eng3')

# Check number of columns and rows
print(eng3_stand.shape, eng3_shooting.shape, eng3_misc.shape)
eng3_agg.shape

(785, 25) (785, 20) (785, 21)


(785, 63)

## Finland

### FIN 1 - Standard Stats, Shooting and Miscellaneous

In [1264]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/43/stats/Veikkausliiga-Stats"
fin1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/43/shooting/Veikkausliiga-Stats"
fin1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/43/misc/Veikkausliiga-Stats"
fin1_misc = scrape_fbref_stats(url3)

In [1266]:
# Aggregate data
fin1_agg = join_player_tables([fin1_stand, fin1_shooting, fin1_misc], 'fin1')

# Check number of columns and rows
print(fin1_stand.shape, fin1_shooting.shape, fin1_misc.shape)
fin1_agg.shape

(296, 25) (296, 20) (296, 21)


(296, 63)

## France

### FRA 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1059]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/13/stats/Ligue-1-Stats"
fra1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/13/shooting/Ligue-1-Stats"
fra1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/13/misc/Ligue-1-Stats"
fra1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/13/passing/Ligue-1-Stats"
fra1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/13/defense/Ligue-1-Stats"
fra1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/13/possession/Ligue-1-Stats"
fra1_pos= scrape_fbref_stats(url6)

In [1060]:
# Aggregate data
fra1_agg = join_player_tables([fra1_stand, fra1_shooting, fra1_misc, fra1_pass, fra1_def, fra1_pos], 'fra1')

# Check number of columns and rows
print(fra1_stand.shape, fra1_shooting.shape, fra1_misc.shape, fra1_pass.shape, fra1_def.shape, fra1_pos.shape)
fra1_agg.shape

(553, 37) (553, 26) (553, 25) (553, 32) (553, 25) (553, 31)


(553, 167)

### FRA 2 - Standard Stats, Shooting and Miscellaneous

In [1062]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/60/stats/Ligue-2-Stats"
fra2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/60/shooting/Ligue-2-Stats"
fra2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/60/misc/Ligue-2-Stats"
fra2_misc = scrape_fbref_stats(url3)

In [1063]:
# Aggregate data
fra2_agg = join_player_tables([fra2_stand, fra2_shooting, fra2_misc], 'fra2')

# Check number of columns and rows
print(fra2_stand.shape, fra2_shooting.shape, fra2_misc.shape)
fra2_agg.shape

(528, 25) (528, 20) (528, 21)


(528, 63)

## Germany

### GER 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1066]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/20/stats/Bundesliga-Stats"
ger1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/20/shooting/Bundesliga-Stats"
ger1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/20/misc/Bundesliga-Stats"
ger1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/20/passing/Bundesliga-Stats"
ger1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/20/defense/Bundesliga-Stats"
ger1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/20/possession/Bundesliga-Stats"
ger1_pos= scrape_fbref_stats(url6)

In [1067]:
# Aggregate data
ger1_agg = join_player_tables([ger1_stand, ger1_shooting, ger1_misc, ger1_pass, ger1_def, ger1_pos], 'ger1')

# Check number of columns and rows
print(ger1_stand.shape, ger1_shooting.shape, ger1_misc.shape, ger1_pass.shape, ger1_def.shape, ger1_pos.shape)
ger1_agg.shape

(492, 37) (492, 26) (492, 25) (492, 32) (492, 25) (492, 31)


(492, 167)

### GER 2 - Standard Stats, Shooting and Miscellaneous

In [1069]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/33/stats/2-Bundesliga-Stats"
ger2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/33/shooting/2-Bundesliga-Stats"
ger2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/33/misc/2-Bundesliga-Stats"
ger2_misc = scrape_fbref_stats(url3)

In [1070]:
# Aggregate data
ger2_agg = join_player_tables([ger2_stand, ger2_shooting, ger2_misc], 'ger2')

# Check number of columns and rows
print(ger2_stand.shape, ger2_shooting.shape, ger2_misc.shape)
ger2_agg.shape

(536, 25) (536, 20) (536, 21)


(536, 63)

### GER 3 - Standard Stats and Miscellaneous

In [1072]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/59/stats/3-Liga-Stats"
ger3_stand = scrape_fbref_stats(url1)
# SHOOTING - NOT AVAILABLE
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/59/misc/3-Liga-Stats"
ger3_misc = scrape_fbref_stats(url3)

In [1073]:
# Aggregate data
ger3_agg = join_player_tables([ger3_stand, ger3_misc], 'ger3')

# Check number of columns and rows
print(ger3_stand.shape, ger3_misc.shape)
ger3_agg.shape

(602, 25) (602, 21)


(602, 45)

## Greece

### GRE 1 - Standard Stats and Miscellaneous

In [1076]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/27/stats/Super-League-Greece-Stats"
gre1_stand = scrape_fbref_stats(url1)
# SHOOTING - NOT AVAILABLE
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/27/misc/Super-League-Greece-Stats"
gre1_misc = scrape_fbref_stats(url3)

In [1077]:
# Aggregate data
gre1_agg = join_player_tables([gre1_stand, gre1_misc], 'gre1')

# Check number of columns and rows
print(gre1_stand.shape, gre1_misc.shape)
gre1_agg.shape

(446, 25) (446, 21)


(446, 45)

## Hungary

### HUN 1 - Standard Stats, Shooting and Miscellaneous

In [1236]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/46/2024-2025/stats/2024-2025-NB-I-Stats"
hun1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/46/2024-2025/shooting/2024-2025-NB-I-Stats"
hun1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/46/2024-2025/misc/2024-2025-NB-I-Stats"
hun1_misc = scrape_fbref_stats(url3)

In [1237]:
# Aggregate data
hun1_agg = join_player_tables([hun1_stand, hun1_shooting, hun1_misc], 'hun1')

# Check number of columns and rows
print(hun1_stand.shape, hun1_shooting.shape, hun1_misc.shape)
hun1_agg.shape

(388, 25) (388, 20) (388, 21)


(388, 63)

## Italy

### ITA 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1080]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/11/stats/Serie-A-Stats"
ita1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/11/shooting/Serie-A-Stats"
ita1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/11/misc/Serie-A-Stats"
ita1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/11/passing/Serie-A-Stats"
ita1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/11/defense/Serie-A-Stats"
ita1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/11/possession/Serie-A-Stats"
ita1_pos= scrape_fbref_stats(url6)

In [1081]:
# Aggregate data
ita1_agg = join_player_tables([ita1_stand, ita1_shooting, ita1_misc, ita1_pass, ita1_def, ita1_pos], 'ita1')

# Check number of columns and rows
print(ita1_stand.shape, ita1_shooting.shape, ita1_misc.shape, ita1_pass.shape, ita1_def.shape, ita1_pos.shape)
ita1_agg.shape

(634, 37) (634, 26) (634, 25) (634, 32) (634, 25) (634, 31)


(634, 167)

### ITA 2 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1083]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/18/stats/Serie-B-Stats"
ita2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/18/shooting/Serie-B-Stats"
ita2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/18/misc/Serie-B-Stats"
ita2_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/18/passing/Serie-B-Stats"
ita2_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/18/defense/Serie-B-Stats"
ita2_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/18/possession/Serie-B-Stats"
ita2_pos= scrape_fbref_stats(url6)

In [1084]:
# Aggregate data
ita2_agg = join_player_tables([ita2_stand, ita2_shooting, ita2_misc, ita2_pass, ita2_def, ita2_pos], 'ita2')

# Check number of columns and rows
print(ita2_stand.shape, ita2_shooting.shape, ita2_misc.shape, ita2_pass.shape, ita2_def.shape, ita2_pos.shape)
ita2_agg.shape

(625, 37) (625, 26) (625, 25) (625, 32) (625, 25) (625, 31)


(625, 167)

## Netherlands

### HOL 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1087]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/23/stats/Eredivisie-Stats"
hol1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/23/shooting/Eredivisie-Stats"
hol1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/23/misc/Eredivisie-Stats"
hol1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/23/passing/Eredivisie-Stats"
hol1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/23/defense/Eredivisie-Stats"
hol1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/23/possession/Eredivisie-Stats"
hol1_pos= scrape_fbref_stats(url6)

In [1088]:
# Aggregate data
hol1_agg = join_player_tables([hol1_stand, hol1_shooting, hol1_misc, hol1_pass, hol1_def, hol1_pos], 'hol1')

# Check number of columns and rows
print(hol1_stand.shape, hol1_shooting.shape, hol1_misc.shape, hol1_pass.shape, hol1_def.shape, hol1_pos.shape)
hol1_agg.shape

(534, 37) (534, 26) (534, 25) (534, 32) (534, 25) (534, 31)


(534, 167)

### HOL 2 - Standard Stats, Shooting and Miscellaneous

In [1090]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/51/stats/Eerste-Divisie-Stats"
hol2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/51/shooting/Eerste-Divisie-Stats"
hol2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/51/misc/Eerste-Divisie-Stats"
hol2_misc = scrape_fbref_stats(url3)

In [1091]:
# Aggregate data
hol2_agg = join_player_tables([hol2_stand, hol2_shooting, hol2_misc], 'hol2')

# Check number of columns and rows
print(hol2_stand.shape, hol2_shooting.shape, hol2_misc.shape)
hol2_agg.shape

(665, 25) (665, 20) (665, 21)


(665, 63)

## Norway

### NOR 1 - Ongoing | Standard Stats, Shooting and Miscellaneous

In [1094]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/28/stats/Eliteserien-Stats"
nor1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/28/shooting/Eliteserien-Stats"
nor1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/28/misc/Eliteserien-Stats"
nor1_misc = scrape_fbref_stats(url3)

In [1095]:
# Aggregate data
nor1_agg = join_player_tables([nor1_stand, nor1_shooting, nor1_misc], 'nor1')

# Check number of columns and rows
print(nor1_stand.shape, nor1_shooting.shape, nor1_misc.shape)
nor1_agg.shape

(368, 25) (368, 20) (368, 21)


(368, 63)

## Poland

### POL 1 - Standard Stats, Shooting and Miscellaneous

In [1098]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/36/2024-2025/stats/2024-2025-Ekstraklasa-Stats"
pol1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/36/2024-2025/shooting/2024-2025-Ekstraklasa-Stats"
pol1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/36/2024-2025/misc/2024-2025-Ekstraklasa-Stats"
pol1_misc = scrape_fbref_stats(url3)

In [1099]:
# Aggregate data
pol1_agg = join_player_tables([pol1_stand, pol1_shooting, pol1_misc], 'pol1')

# Check number of columns and rows
print(pol1_stand.shape, pol1_shooting.shape, pol1_misc.shape)
pol1_agg.shape

(552, 25) (552, 20) (552, 21)


(552, 63)

## Portugal

### POR 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1102]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/32/stats/Primeira-Liga-Stats"
por1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/32/shooting/Primeira-Liga-Stats"
por1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/32/misc/Primeira-Liga-Stats"
por1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/32/passing/Primeira-Liga-Stats"
por1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/32/defense/Primeira-Liga-Stats"
por1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/32/possession/Primeira-Liga-Stats"
por1_pos= scrape_fbref_stats(url6)

In [1103]:
# Aggregate data
por1_agg = join_player_tables([por1_stand, por1_shooting, por1_misc, por1_pass, por1_def, por1_pos], 'por1')

# Check number of columns and rows
print(por1_stand.shape, por1_shooting.shape, por1_misc.shape, por1_pass.shape, por1_def.shape, por1_pos.shape)
por1_agg.shape

(585, 37) (585, 26) (585, 25) (585, 32) (585, 25) (585, 31)


(585, 167)

## Romania

### ROM 1 - Standard Stats, Shooting and Miscellaneous

In [1230]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/47/2024-2025/stats/2024-2025-Liga-I-Stats"
rom1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/47/2024-2025/shooting/2024-2025-Liga-I-Stats"
rom1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/47/2024-2025/misc/2024-2025-Liga-I-Stats"
rom1_misc = scrape_fbref_stats(url3)

In [1231]:
# Aggregate data
rom1_agg = join_player_tables([rom1_stand, rom1_shooting, rom1_misc], 'rom1')

# Check number of columns and rows
print(rom1_stand.shape, rom1_shooting.shape, rom1_misc.shape)
rom1_agg.shape

(574, 25) (574, 20) (574, 21)


(574, 63)

## Russia

### RUS 1 - Standard Stats, Shooting and Miscellaneous

In [1269]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/30/2024-2025/stats/2024-2025-Russian-Premier-League-Stats"
rus1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/30/2024-2025/shooting/2024-2025-Russian-Premier-League-Stats"
rus1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/30/2024-2025/misc/2024-2025-Russian-Premier-League-Stats"
rus1_misc = scrape_fbref_stats(url3)

In [1270]:
# Aggregate data
rus1_agg = join_player_tables([rus1_stand, rus1_shooting, rus1_misc], 'rus1')

# Check number of columns and rows
print(rus1_stand.shape, rus1_shooting.shape, rus1_misc.shape)
rus1_agg.shape

(477, 25) (477, 20) (477, 21)


(477, 63)

## Scotland

### SCO 1 - Standard Stats, Shooting and Miscellaneous

In [1106]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/40/stats/Scottish-Premiership-Stats"
sco1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/40/shooting/Scottish-Premiership-Stats"
sco1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/40/misc/Scottish-Premiership-Stats"
sco1_misc = scrape_fbref_stats(url3)

In [1107]:
# Aggregate data
sco1_agg = join_player_tables([sco1_stand, sco1_shooting, sco1_misc], 'sco1')

# Check number of columns and rows
print(sco1_stand.shape, sco1_shooting.shape, sco1_misc.shape)
sco1_agg.shape

(372, 25) (372, 20) (372, 21)


(372, 63)

## Serbia

### SER 1 - Standard Stats, Shooting and Miscellaneous

In [1110]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/54/2024-2025/stats/2024-2025-Serbian-SuperLiga-Stats"
ser1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/54/2024-2025/shooting/2024-2025-Serbian-SuperLiga-Stats"
ser1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/54/2024-2025/misc/2024-2025-Serbian-SuperLiga-Stats"
ser1_misc = scrape_fbref_stats(url3)

In [1111]:
# Aggregate data
ser1_agg = join_player_tables([ser1_stand, ser1_shooting, ser1_misc], 'ser1')

# Check number of columns and rows
print(ser1_stand.shape, ser1_shooting.shape, ser1_misc.shape)
ser1_agg.shape

(579, 25) (579, 20) (579, 21)


(579, 63)

## Spain

### SPA 1 - Standard Stats, Shooting, Miscellaneous, Passing, Defensive and Possession

In [1114]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/12/stats/La-Liga-Stats"
spa1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/12/shooting/La-Liga-Stats"
spa1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/12/misc/La-Liga-Stats"
spa1_misc = scrape_fbref_stats(url3)
# PASSING
url4= "https://fbref.com/en/comps/12/passing/La-Liga-Stats"
spa1_pass = scrape_fbref_stats(url4)
# DEFENSIVE
url5 = "https://fbref.com/en/comps/12/defense/La-Liga-Stats"
spa1_def= scrape_fbref_stats(url5)
# POSSESSION
url6 = "https://fbref.com/en/comps/12/possession/La-Liga-Stats"
spa1_pos= scrape_fbref_stats(url6)

In [1115]:
# Aggregate data
spa1_agg = join_player_tables([spa1_stand, spa1_shooting, spa1_misc, spa1_pass, spa1_def, spa1_pos], 'spa1')

# Check number of columns and rows
print(spa1_stand.shape, spa1_shooting.shape, spa1_misc.shape, spa1_pass.shape, spa1_def.shape, spa1_pos.shape)
spa1_agg.shape

(601, 37) (601, 26) (601, 25) (601, 32) (601, 25) (601, 31)


(601, 167)

### SPA 2 - Standard Stats, Shooting and Miscellaneous

In [1117]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/17/stats/Segunda-Division-Stats"
spa2_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/17/shooting/Segunda-Division-Stats"
spa2_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/17/misc/Segunda-Division-Stats"
spa2_misc = scrape_fbref_stats(url3)

In [1118]:
# Aggregate data
spa2_agg = join_player_tables([spa2_stand, spa2_shooting, spa2_misc], 'spa2')

# Check number of columns and rows
print(spa2_stand.shape, spa2_shooting.shape, spa2_misc.shape)
spa2_agg.shape

(680, 25) (680, 20) (680, 21)


(680, 63)

## Sweden

### SWE 1 | Ongoing - Standard Stats, Shooting and Miscellaneous

In [1121]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/29/stats/Allsvenskan-Stats"
swe1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/29/shooting/Allsvenskan-Stats"
swe1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/29/misc/Allsvenskan-Stats"
swe1_misc = scrape_fbref_stats(url3)

In [1122]:
# Aggregate data
swe1_agg = join_player_tables([swe1_stand, swe1_shooting, swe1_misc], 'swe1')

# Check number of columns and rows
print(swe1_stand.shape, swe1_shooting.shape, swe1_misc.shape)
swe1_agg.shape

(394, 25) (394, 20) (394, 21)


(394, 63)

## Switzerland

### SUI 1 - Standard Stats, Shooting and Miscellaneous

In [1125]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/57/2024-2025/stats/2024-2025-Swiss-Super-League-Stats"
swi1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/57/2024-2025/shooting/2024-2025-Swiss-Super-League-Stats"
swi1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/57/2024-2025/misc/2024-2025-Swiss-Super-League-Stats"
swi1_misc = scrape_fbref_stats(url3)

In [1126]:
# Aggregate data
swi1_agg = join_player_tables([swi1_stand, swi1_shooting, swi1_misc], 'swi1')

# Check number of columns and rows
print(swi1_stand.shape, swi1_shooting.shape, swi1_misc.shape)
swi1_agg.shape

(380, 25) (380, 20) (380, 21)


(380, 63)

## Turkey

### TUR 1 - Standard Stats, Shooting and Miscellaneous

In [1129]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/26/stats/Super-Lig-Stats"
tur1_stand = scrape_fbref_stats(url1)
# SHOOTING
url2 = "https://fbref.com/en/comps/26/shooting/Super-Lig-Stats"
tur1_shooting = scrape_fbref_stats(url2)
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/26/misc/Super-Lig-Stats"
tur1_misc = scrape_fbref_stats(url3)

In [1130]:
# Aggregate data
tur1_agg = join_player_tables([tur1_stand, tur1_shooting, tur1_misc], 'tur1')

# Check number of columns and rows
print(tur1_stand.shape, tur1_shooting.shape, tur1_misc.shape)
tur1_agg.shape

(598, 25) (598, 20) (598, 21)


(598, 63)

## Ukraine

### UKR 1 - Standard Stats and Miscellaneous

In [1273]:
# STANDARD STATS
url1 = "https://fbref.com/en/comps/39/stats/Ukrainian-Premier-League-Stats"
ukr1_stand = scrape_fbref_stats(url1)
# SHOOTING - NOT AVAILABLE
# MISCELLANEOUS
url3 = "https://fbref.com/en/comps/39/misc/Ukrainian-Premier-League-Stats"
ukr1_misc = scrape_fbref_stats(url3)

In [1274]:
# Aggregate data
ukr1_agg = join_player_tables([ukr1_stand, ukr1_misc], 'ukr1')

# Check number of columns and rows
print(ukr1_stand.shape, ukr1_misc.shape)
ukr1_agg.shape

(521, 25) (521, 21)


(521, 45)

# 3. All Leagues Consolidation and Data Cleaning

In [1202]:
def standardize_and_concat(dfs, fill_value='-'):
    """
    Standardizes column sets across multiple DataFrames and concatenates them.

    Parameters:
    - dfs: list of pandas DataFrames
    - fill_value: value used to fill missing columns (default: '-')

    Returns:
    - A single concatenated DataFrame with aligned and ordered columns
    """
    # Step 1: Get full set of all column names across all dataframes
    all_columns = set()
    for df in dfs:
        all_columns.update(df.columns)

    # Step 2: Determine desired column order:
    # Start with columns from the first dataframe
    base_order = list(dfs[0].columns)
    # Append any new columns from others (not in the first one)
    extra_columns = [col for col in all_columns if col not in base_order]
    final_columns = base_order + sorted(extra_columns)

    # Step 3: Reindex each dataframe to have all columns (fill missing with '-')
    standardized_dfs = [df.reindex(columns=all_columns, fill_value=fill_value) for df in dfs]

    # Step 4: Concatenate all dataframes
    combined_df = pd.concat(standardized_dfs, ignore_index=True)

    # Step 5: Add all missing columns (if any) in one go
    missing_cols = [col for col in final_columns if col not in combined_df.columns]
    if missing_cols:
        missing_df = pd.DataFrame({col: fill_value for col in missing_cols}, index=combined_df.index)
        combined_df = pd.concat([combined_df, missing_df], axis=1)

    # Step 6: Reorder columns
    combined_df = combined_df[final_columns]

    return combined_df


In [1279]:
all_leagues = [eng1_agg, aus1_agg, bel1_agg, bel2_agg, bul1_agg, cro1_agg, cze1_agg, 
               den1_agg, eng2_agg, eng3_agg, fin1_agg, fra1_agg, fra2_agg, ger1_agg, 
               ger2_agg, ger3_agg, gre1_agg, hun1_agg, ita1_agg, ita2_agg, hol1_agg, 
               hol2_agg, nor1_agg, pol1_agg, por1_agg, rom1_agg, rus1_agg, sco1_agg, 
               ser1_agg, spa1_agg, spa2_agg, swe1_agg, swi1_agg, tur1_agg, ukr1_agg]

In [1281]:
df_complete = standardize_and_concat(all_leagues)
df_complete.shape

(18224, 169)

In [1283]:
cols_to_drop = ['Nation_2','Pos_2','Squad_2','Age_2','Born_2','Matches_2',
                'Nation_3','Pos_3','Squad_3','Age_3','Born_3','Matches_3',
                'Nation_4','Pos_4','Squad_4','Age_4','Born_4','Matches_4',
                'Nation_5','Pos_5','Squad_5','Age_5','Born_5','Matches_5',
                'Nation_6','Pos_6','Squad_6','Age_6','Born_6','Matches_6',
                '90s_3','90s_4','90s_5','90s_6',
                'Rk','Matches','Expected_xG_2','Expected_npxG_2',
                'Performance_CrdR_2','Performance_CrdY_2',
                'Performance_CrdY_3','Performance_CrdR_3']
df_complete.drop(columns=cols_to_drop, inplace = True)
df_complete.shape

(18224, 127)

In [1285]:
df_complete.to_excel(r"C:\Users\pedro\OneDrive\Escritorio\Projetos\Football Scout/df_complete.xlsx", index=False)